In [ ]:
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import h5py

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import plotly.offline as py

In [ ]:
%matplotlib inline

In [ ]:
from dautil.plot import plot_column_slider

In [ ]:
%time from tail.analysis.container import Spectra, FSky, NullSpectra

In [ ]:
%time from tail.analysis.likelihood import ComputeLikelihood

In [ ]:
save = True
transform_real = True

In [ ]:
basedir = Path('/scratch/largepatch_new')
path = basedir / 'high_1_0_2.hdf5'
path_mask = basedir / 'masks.hdf5'

In [ ]:
path_cache = basedir / 'high_1_0_2_spectra_final.hdf5'

In [ ]:
if transform_real:
    with h5py.File(path_cache, 'r') as f:
        thetas_map = f['/full/spectra/100_4300_100/thetas_map'][:]
    print(1. / thetas_map[1], 1. / thetas_map[2], thetas_map[3] * (180. / np.pi), thetas_map[4] * (10800. / np.pi)**2)

In [ ]:
outdir = Path('media')

In [ ]:
if save:
    outdir.mkdir(exist_ok=True)

In [ ]:
# select spectra
bin_width = 100

In [ ]:
# narrow l-range for science
l_min = 600
l_max = 3000
l_T_cutoff = 1600

In [ ]:
f_sky = FSky.load(path_mask)

In [ ]:
%time spectra = Spectra.load(path_cache, bin_width, full=True, path_theory_spectra=path)

In [ ]:
%time spectra = spectra.slicing_l(l_min, l_max, ascontiguousarray=True)

In [ ]:
%time likelihood_computer = ComputeLikelihood.load(spectra, f_sky, l_T_cutoff=l_T_cutoff)

In [ ]:
for i in likelihood_computer.map:
    print(i)

In [ ]:
likelihood_computer.LR_null

In [ ]:
%time likelihood_computer.LR_fiducial

In [ ]:
%%time
likelihood_computer.plot_mcmc()
if save:
    plt.savefig(outdir / 'pair-scatter.png', dpi=180)

In [ ]:
likelihood_computer.plot_mcmc(kind='grid')
if save:
    plt.savefig(outdir / 'pair-kde.svg')

In [ ]:
likelihood_computer.mcmc_ci()

In [ ]:
likelihood_computer.map[0] / np.std(likelihood_computer.thetas[:, 0])

In [ ]:
df_sum = likelihood_computer.df_thetas.describe()

In [ ]:
df_sum

In [ ]:
if save:
    df_sum.to_csv(outdir / 'likelihood.csv', float_format='%.3g')

Calibration factors combined with the iterative `theta_map`

In [ ]:
a_T = np.reciprocal(thetas_map[1] * likelihood_computer.thetas[:, 1])
a_T.mean(), np.std(a_T, ddof=1)

In [ ]:
a_E = np.reciprocal(thetas_map[2] * likelihood_computer.thetas[:, 2])
a_E.mean(), np.std(a_E, ddof=1)

In [ ]:
theta_degree = (thetas_map[3] + likelihood_computer.thetas[:, 3]) * (180. / np.pi)
theta_degree.mean(), np.std(theta_degree, ddof=1)

In [ ]:
sigma_sq_arcmin = (thetas_map[4] + likelihood_computer.thetas[:, 4]) * (10800. / np.pi)**2
sigma_sq_arcmin.mean(), np.std(sigma_sq_arcmin, ddof=1)

# Error Propagation

In [ ]:
spectra = likelihood_computer.err_mcmc_to_spectra()

In [ ]:
df_pte = pd.DataFrame(spectra.pte[:, 0, 0], columns=['MC chi-sq PTE'], index=spectra.spectra)

In [ ]:
df_pte

In [ ]:
if save and transform_real:
    df_pte.to_csv(outdir / 'pte.csv', float_format='%.3g')

In [ ]:
spectra.err_mcmc.shape

In [ ]:
fig = spectra.plot_err_mcmc()

In [ ]:
if save and transform_real:
    py.plot(fig, filename=str(outdir / 'err_mcmc.html'), include_plotlyjs='cdn', include_mathjax='cdn')

In [ ]:
fig

In [ ]:
fig2 = spectra.plot_spectra()

In [ ]:
if save and transform_real:
    py.plot(fig2, filename=str(outdir / 'spectra_with_err_mcmc.html'), include_plotlyjs='cdn', include_mathjax='cdn')

In [ ]:
fig2